In [1]:
import numpy as np
import pandas as pd

In [2]:
train_file = './assignment-pgm-1/train.csv'
test_file = './assignment-pgm-1/test.csv'
submit_file = './assignment-pgm-1/submission.csv'

### Since all are binary random variables

In [3]:
d=2

In [4]:
def NBfit(train_file):
    
    '''
    Function to fit the data into naive bayes model and calculate CPDs
    '''
    
    train_data = pd.read_csv(train_file)
    print("Overview  of Training data : \n {}".format(train_data.head()))
    
    # separation of data into class 0 and 1
    ind_0 = train_data['class']==0
    # class labels
    actual_classes = train_data['class']
    del train_data['class']
    #indexes of class 1.
    ind_1 = ~ind_0
    tr_data_0,tr_data_1 = train_data[ind_0].values,train_data[ind_1].values
    
    #calculation of probabilities
    p_01  = (np.sum(tr_data_0,axis=0)+1)/(len(tr_data_0)+d)            #feature = 1 given class 0
    p_11 = (np.sum(tr_data_1,axis=0)+1)/(len(tr_data_0)+d)             #feature = 1 given class 1

    cpd_given_0 = np.log(p_01)
    cpd_given_1 = np.log(p_11)

    feat_cpds = pd.DataFrame([cpd_given_0,cpd_given_1],columns=train_data.columns)
    
    return feat_cpds,train_data,actual_classes

In [5]:
def BNfit(train_file):
    
    '''
    Function to fit the data into Bayesian Network
    '''
    
    train_data = pd.read_csv(train_file)
    print("Overview  of Training data : \n {}".format(train_data.head()))
    
    ind_0 = train_data['class']==0
    actual_classes = train_data['class']
    del train_data['class']
    ind_1 = ~ind_0
    tr_data_0,tr_data_1 = train_data[ind_0],train_data[ind_1]

    #parent node for special dependency given
    parents = ['V16']
    # children nodes of the parent above
    children = ['V8','V9']
    
    #indexes 
    ind_0_0 = (tr_data_0[parents[0]] == 0)            #v16 -> 0 and class 0
    ind_0_1 = ~ind_0_0                                            #v16 -> 1 and class 0
    ind_1_0 = (tr_data_1[parents[0]] == 0)            #v16 -> 0 and class 1
    ind_1_1 = ~ind_1_0                                            #v16 -> 1 and class 1

    feat_dep_cpds = []
    
    #calc of log probabs for 4 different params for v8 and v9.
    feat_dep_cpds.append(np.log((np.sum(tr_data_0[children][ind_0_0],axis=0)+1)/(len(tr_data_0[children][ind_0_0])+d)))
    feat_dep_cpds.append(np.log((np.sum(tr_data_0[children][ind_0_1],axis=0)+1)/(len(tr_data_0[children][ind_0_1])+d)))
    feat_dep_cpds.append(np.log((np.sum(tr_data_1[children][ind_1_0],axis=0)+1)/(len(tr_data_1[children][ind_1_0])+d)))
    feat_dep_cpds.append(np.log((np.sum(tr_data_1[children][ind_1_1],axis=0)+1)/(len(tr_data_1[children][ind_1_1])+d)))
      
    # deleting the children node from dataframe to find normal probab for other features.
    for node in children:
        del tr_data_0[node]
        del tr_data_1[node]
        
    #log probab for features other than v8,v9
    cpd_given_0 = np.log((np.sum(tr_data_0.values,axis=0)+1)/(len(tr_data_0)+d))
    cpd_given_1 = np.log((np.sum(tr_data_1.values,axis=0)+1)/(len(tr_data_1)+d))
    
    cols = list(train_data.columns)
    for child in children:
        del cols[cols.index(child)]
        
    feat_normal_cpds = pd.DataFrame([cpd_given_0,cpd_given_1],columns=cols)
    feat_dep_cpds = pd.DataFrame(feat_dep_cpds,columns=children)
    
    return feat_normal_cpds,feat_dep_cpds,children,parents,train_data,actual_classes

In [6]:
def NBeval(file,feat_cpds,submit_file=None):
    
    '''
    Function to predict the class of given sample using naive bayes cpds.
    '''
    
    data = pd.read_csv(file)
    pred_classes = pd.DataFrame(index=np.arange(len(data)),columns=['id','class'])
     
    if 'id' in data.columns: 
        del data['id']
    else:
        del data['class']    

    for i in range(data.shape[0]):

        pr_sample_0 = np.log(0.5)
        pr_sample_1 = np.log(0.5)
        
        for j,x in enumerate(data.iloc[i,:]):
            if (x==1): 
                pr_sample_0 += feat_cpds.iloc[0,j]
                pr_sample_1 += feat_cpds.iloc[1,j]
            else:
                pr_sample_0 += np.log(1-np.exp(feat_cpds.iloc[0,j]))
                pr_sample_1 += np.log(1-np.exp(feat_cpds.iloc[1,j]))

            pred_classes.iloc[i,0] = i+1
            
            #assigning class labels based on log probabs
            # Break tie when they are equal
            if(pr_sample_1>=pr_sample_0):
                pred_classes.iloc[i,1] = 1
            else:
                pred_classes.iloc[i,1] = 0
    
    if(submit_file is not None):
        pred_classes.to_csv(path_or_buf=submit_file,index=False)
    
    return pred_classes

In [7]:
def BNeval(file,feat_normal_cpds,feat_dep_cpds,children,parents,submit_file=None):

    '''
    Function to predict the class of given sample using Bayesian network cpds.
    feat_normal_cpds - stores log probab of features other than v8 , v9
    feat_dep_cpds  - stores log prob for v8 and v9.
    '''
    
    data = pd.read_csv(file)
    pred_classes = pd.DataFrame(index=np.arange(len(data)),columns=['id','class'])
    
    if 'id' in data.columns: 
        del data['id']
    else:
        del data['class']
        
    for i in range(data.shape[0]):

        #prior probabilities
        pr_sample_0 = np.log(0.5)
        pr_sample_1 = np.log(0.5)
        
        for j,x in enumerate(data.iloc[i,:]):
            curr_col = data.columns[j]

            if(curr_col in children):

                if(data.loc[i,parents[0]]==1):
                    if (x==1):
                        pr_sample_0 += feat_dep_cpds.loc[1,curr_col]
                        pr_sample_1  += feat_dep_cpds.loc[3,curr_col]
                    else:
                        pr_sample_0 += np.log(1-np.exp(feat_dep_cpds.loc[1,curr_col]))
                        pr_sample_1  += np.log(1-np.exp(feat_dep_cpds.loc[3,curr_col]))
                else:
                    if(x==1):
                        pr_sample_0 += (feat_dep_cpds.loc[0,curr_col])
                        pr_sample_1 += feat_dep_cpds.loc[2,curr_col]
                    else:
                        pr_sample_0 += np.log(1-np.exp(feat_dep_cpds.loc[0,curr_col]))
                        pr_sample_1 += np.log(1-np.exp(feat_dep_cpds.loc[2,curr_col]))
            else:
                if(x==1):
                    pr_sample_0 += feat_normal_cpds.loc[0,curr_col]
                    pr_sample_1 += feat_normal_cpds.loc[1,curr_col]
                else:
                    pr_sample_0 += np.log(1-np.exp(feat_normal_cpds.loc[0,curr_col]))
                    pr_sample_1 += np.log(1-np.exp(feat_normal_cpds.loc[1,curr_col]))
                      
            pred_classes.iloc[i,0] = i+1

            # Break tie when they are equal
            if(pr_sample_1>=pr_sample_0):
                pred_classes.iloc[i,1] = 1
            else:
                pred_classes.iloc[i,1] = 0

        if (submit_file is not None):
            pred_classes.to_csv(path_or_buf=submit_file,index=False)
            
    return pred_classes

In [8]:
def accuracy(actual_classes,pred_classes):
    '''
    Function to calculate accuracy of a classifier when actual class labels are known.
    '''
    train_acc = np.sum(actual_classes == pred_classes)/(len(actual_classes))*100
    print(train_acc)

## Naive Bayes Implementation
* change 'train_file'  into 'test_file' to run the test data

In [9]:
feat_cpds,train_data,actual_classes = NBfit(train_file)
pred_classes = NBeval(train_file,feat_cpds=feat_cpds,submit_file=None)
print("Predicted classes distribution : \n{}" .format(pred_classes['class'].value_counts()))
nb_pred_classes = pred_classes['class'].values

Overview  of Training data : 
    class  V1  V2  V3  V4  V5  V6  V7  V8  V9 ...   V13  V14  V15  V16  V17  \
0      1   0   0   0   1   0   0   0   1   1 ...     1    1    0    0    0   
1      1   0   0   1   1   0   0   0   1   1 ...     1    1    0    0    0   
2      1   1   0   1   0   1   0   0   1   0 ...     1    0    0    0    0   
3      1   0   0   0   0   0   0   0   0   0 ...     0    0    0    0    0   
4      1   0   0   0   0   0   0   0   1   0 ...     1    0    1    1    0   

   V18  V19  V20  V21  V22  
0    0    0    0    0    0  
1    0    0    0    0    1  
2    0    0    0    0    0  
3    0    0    1    1    1  
4    0    0    0    0    0  

[5 rows x 23 columns]
Predicted classes distribution : 
0    49
1    31
Name: class, dtype: int64


In [10]:
accuracy(actual_classes,pred_classes=nb_pred_classes)

78.75


## Bayesian Network Implementation

In [11]:
feat_normal_cpds,feat_dep_cpds,children,parents,train_data,actual_classes = BNfit(train_file)
bn_pred_classes = BNeval(train_file,feat_normal_cpds,feat_dep_cpds,children=children,parents=parents,submit_file=None)
print("Predicted classes distribution : \n{}" .format(bn_pred_classes['class'].value_counts()))

Overview  of Training data : 
    class  V1  V2  V3  V4  V5  V6  V7  V8  V9 ...   V13  V14  V15  V16  V17  \
0      1   0   0   0   1   0   0   0   1   1 ...     1    1    0    0    0   
1      1   0   0   1   1   0   0   0   1   1 ...     1    1    0    0    0   
2      1   1   0   1   0   1   0   0   1   0 ...     1    0    0    0    0   
3      1   0   0   0   0   0   0   0   0   0 ...     0    0    0    0    0   
4      1   0   0   0   0   0   0   0   1   0 ...     1    0    1    1    0   

   V18  V19  V20  V21  V22  
0    0    0    0    0    0  
1    0    0    0    0    1  
2    0    0    0    0    0  
3    0    0    1    1    1  
4    0    0    0    0    0  

[5 rows x 23 columns]
Predicted classes distribution : 
0    47
1    33
Name: class, dtype: int64


In [12]:
accuracy(actual_classes=actual_classes,pred_classes=bn_pred_classes['class'].values)

81.25


In [13]:
np.exp(feat_dep_cpds)

,V8,V9
0,0.097561,0.097561
1,0.666667,0.666667
2,0.413793,0.206897
3,0.533333,0.466667


In [14]:
np.exp(feat_normal_cpds)

,V1,V2,V3,V4,V5,V6,V7,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
0,0.285714,0.095238,0.166667,0.119048,0.261905,0.095238,0.142857,0.214286,0.095238,0.142857,0.142857,0.119048,0.047619,0.047619,0.023810,0.023810,0.142857,0.142857,0.119048,0.190476
1,0.452381,0.261905,0.380952,0.333333,0.357143,0.190476,0.404762,0.380952,0.333333,0.380952,0.571429,0.309524,0.142857,0.333333,0.214286,0.166667,0.261905,0.333333,0.380952,0.476190


## Finding a better Bayesian Network
To learn the structure from the given data,
* We can find the correlation matrix of all the features.
* Take the top $k$ correlated feature pairs, where $k$ is hyperparameter can be found using cross validation by splitting train data.
* Since searching over the exponential no of possible BN structures blindly is not efficient, so we restrict this by using the correlation of features.
* But since we don't know the causation still, we have to randomly choose which one has to be parent among the selected pairs.
* By experimenting in this manner, we can achieve a better result than we got earlier. Due to time constraint, I couldn't implement it, but this is the idea I have in mind.

In [15]:
cov_mat = train_data.cov()

### Correlation Matrix of features

In [16]:
cov_mat

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
V1,0.234019,0.028956,0.042880,0.010601,0.193671,0.030063,0.030222,0.025633,0.019778,0.185601,...,0.036076,0.040506,0.010443,0.024367,0.026582,0.010443,0.095728,0.043987,0.026741,0.007278
V2,0.028956,0.137816,0.020095,-0.009652,0.039241,0.055380,0.096044,0.018038,0.019778,0.028639,...,0.005696,0.017722,-0.012342,0.059810,0.059494,0.050949,0.045095,0.026266,0.036867,0.035127
V3,0.042880,0.020095,0.196044,0.082753,0.008861,-0.007911,0.031487,0.142089,0.051424,0.037500,...,0.134810,0.073418,0.030696,0.067405,0.024051,0.030696,0.000791,0.016139,0.101424,0.052848
V4,0.010601,-0.009652,0.082753,0.169462,-0.026582,-0.014241,-0.005854,0.054747,0.086234,0.001424,...,0.076582,0.096203,0.034494,0.050949,0.003797,0.021835,-0.002373,0.014873,0.050158,0.043987
V5,0.193671,0.039241,0.008861,-0.026582,0.212658,0.037975,0.034177,-0.007595,0.006329,0.153165,...,0.007595,0.027848,-0.010127,0.010127,0.032911,0.002532,0.082278,0.045570,0.003797,0.002532
V6,0.030063,0.055380,-0.007911,-0.014241,0.037975,0.110759,0.017405,-0.009494,0.001582,0.014241,...,-0.018987,0.012658,-0.009494,0.003165,0.050633,0.015823,0.064873,0.034810,-0.017405,-0.003165
V7,0.030222,0.096044,0.031487,-0.005854,0.034177,0.017405,0.196044,0.028165,0.026108,0.037500,...,0.020886,0.022785,0.005380,0.054747,0.049367,0.043354,0.038766,0.003481,0.088766,0.065506
V8,0.025633,0.018038,0.142089,0.054747,-0.007595,-0.009494,0.028165,0.201899,0.061709,0.021203,...,0.117722,0.045570,0.042405,0.052532,0.010127,0.017089,-0.001582,0.000633,0.073101,0.010759
V9,0.019778,0.019778,0.051424,0.086234,0.006329,0.001582,0.026108,0.061709,0.154272,0.021361,...,0.047468,0.126582,0.036392,0.055380,0.006329,0.011076,0.027690,0.033228,0.043513,0.014241
V10,0.185601,0.028639,0.037500,0.001424,0.153165,0.014241,0.037500,0.021203,0.021361,0.207437,...,0.037342,0.030380,0.003481,0.012342,0.021519,-0.009177,0.071994,0.035759,0.032120,0.006646


### Top correlated features w.r.t V1

In [20]:
train_data.columns[np.argsort(cov_mat.values,axis=-1)[0][::-1]]

Index(['V1', 'V5', 'V10', 'V19', 'V11', 'V12', 'V20', 'V3', 'V14', 'V13', 'V7',
       'V6', 'V2', 'V21', 'V17', 'V8', 'V16', 'V9', 'V4', 'V18', 'V15', 'V22'],
      dtype='object')